In [1]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.6 MB/s eta 0:00:00


In [5]:
import csv
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.namespace import XSD

ont_19 = Namespace("http://www.semanticweb.org/addalin/ontologies/2024/3/untitled-ontology-19/")
ont_14 = Namespace("http://www.semanticweb.org/addalin/ontologies/2024/3/untitled-ontology-14/")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

def country_name_by_code(code):
  ans = ""
  with open("country_code.csv", "r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      if code == row["IOC"]:
        ans = row["Country"]
  return ans


# Load ontology
g = Graph()
g.parse("ontology_merged.ttl", format="turtle")

hasCountry_Name = ont_19.hasCountry_Name
g.add((hasCountry_Name, RDF.type, OWL.ObjectProperty))

Field_Goals_Percentage = ont_14.Field_Goals_Percentage
g.add((Field_Goals_Percentage, RDF.type, OWL.ObjectProperty))

Free_Throws_Percentage = ont_14.Free_Throws_Percentage
g.add((Free_Throws_Percentage, RDF.type, OWL.ObjectProperty))

with open("Boxscore.csv", "r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:

      plusminus_uri = URIRef(ont_14[str(row["PM"])])
      g.add((plusminus_uri, hasCountry_Name, Literal(country_name_by_code(row["Team"]), datatype=XSD.string)))

      FieldGoals_team_uri = URIRef(ont_14[str(row["FG"].replace("/", "by"))])
      g.add((FieldGoals_team_uri, Field_Goals_Percentage,Literal(str(row["FG%"]))))

      FreeThrow_team_uri = URIRef(ont_14[str(row["FT"].replace("/", "by"))])
      g.add((FreeThrow_team_uri, Free_Throws_Percentage,Literal(str(row["FT%"]))))

g.serialize(destination="ontology_merged_populated.ttl", format="turtle")



<Graph identifier=N7ad6c99d02e4484cbf567c42ea87aa2c (<class 'rdflib.graph.Graph'>)>